# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import json
import os

# Import API key
from api_keys import g_key

In [2]:
# Configure gmaps
gmaps.configure(api_key = g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
# Create aiport dataframe
df = pd.read_csv('weather.csv')

In [4]:
df

,City,Lat,Lon,Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Pacific Grove,36.6177,-121.9166,290.29,83,20,0.45,US,1628383718
1,Hermanus,-34.4187,19.2345,281.93,67,85,2.00,ZA,1628383612
2,Dhārchula,29.8500,80.5333,296.31,90,99,0.59,IN,1628384033
3,Lompoc,34.6391,-120.4579,293.24,73,1,5.14,US,1628383697
4,Ponta do Sol,32.6667,-17.1000,294.60,88,59,2.36,PT,1628383391
...,...,...,...,...,...,...,...,...,...
563,Slave Lake,55.2834,-114.7690,288.14,67,90,9.77,CA,1628384267
564,Boissevain,49.2306,-100.0559,299.43,21,5,3.10,CA,1628384268
565,Itabira,-19.6192,-43.2269,284.06,96,94,0.59,BR,1628384268
566,Sumenep,-7.0167,113.8667,302.00,72,100,6.85,ID,1628384268


### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
df_ideal = df[(df["Temp"] >= 297)
              &(df["Temp"] < 300)
              & (df["Wind Speed"] < 10)
              & (df["Cloudiness"] <= 10)
              & (df["Humidity"] > 30)
              & (df["Humidity"] < 40)]

In [6]:
df_ideal

,City,Lat,Lon,Temp,Humidity,Cloudiness,Wind Speed,Country,Date
431,Turaif,31.6725,38.6637,299.29,34,0,1.54,SA,1628384207
514,Villamontes,-21.2500,-63.5000,297.56,37,0,2.58,BO,1628384244


In [7]:
locations = df_ideal[["Lat", "Lon"]]
locations

,Lat,Lon
431,31.6725,38.6637
514,-21.2500,-63.5000


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [8]:
humidity_list = df_ideal["Humidity"].astype(float)
humidity_list

431    34.0
514    37.0
Name: Humidity, dtype: float64

In [9]:
cities = df_ideal["City"]
countries = df_ideal["Country"]

In [12]:
# Plot Heatmap
fig = gmaps.figure(map_type="HYBRID")

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity_list, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [13]:
hotel_name_list = []
for lat, lon in zip(df_ideal["Lat"], df_ideal["Lon"]):
    
    url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={lat},{lon}&radius=5000&type=hotel&key={g_key}"
    response = requests.get(url)
    d = json.loads(response.text)
    hotel_name = d["results"][0]["name"]
    hotel_name_list.append(hotel_name)

In [14]:
hotel_name_list

['Turaif', 'Villamontes']

In [15]:
# fig = gmaps.figure()
markers = gmaps.marker_layer(locations,
    info_box_content=[f"Hotel: {hotel}, city:{city} country:{country}" for hotel, city, country in zip(hotel_name_list, cities, countries)])
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))